In [1]:
import requests
from bs4 import BeautifulSoup
import json

## Extracting each animal url from our file

In [2]:
animals_urls = []

with open("animals'_urls.txt", mode = "r") as f:
    animals_urls = f.read().split("\n")

## Request for each url

In [3]:
def request_soup(url):
    animal_request = requests.get(url)
    animal_bs = BeautifulSoup(animal_request.text, "html.parser")
    return animal_bs

## Crawling

In [6]:
def crawling(animal_bs):
    animals_dictionary = {}
    
    ''' 
    Animal's name 
    '''
    animal_name = animal_bs("h1", attrs = {"class": "a-h1"})[0].getText().replace("\n", "").replace("  ", "")
    animals_dictionary["Name"] = animal_name
    
    '''
    NTTP - Next to the picture
    '''
    nttp_names = [name.getText().replace("\n", "").replace(" ", "") for name in animal_bs("div", attrs = {"class": "s-char-kinds__attr"})]
    nttp_data = [data.getText() for data in animal_bs("a", attrs = {"class": "s-char-kinds__name"})]
    nttp_data.append(animal_bs("div", attrs = {"class": "col-8 s-char-kinds__name unactive"})[0].getText().replace("\n", "").replace("  ", ""))
    
    for i in range(len(nttp_names)):
        animals_dictionary[nttp_names[i]] = nttp_data[i]
    
    '''
    UTP - Under the picture
    '''
    utp_names = [name.getText() for name in animal_bs("div", attrs = {"class": "s-char-char__name"})]
    utp_data = [data.getText() for data in animal_bs("div", attrs = {"class": "s-char-char__num"})]
    
    for i in range(len(utp_names)):
        animals_dictionary[utp_names[i]] = utp_data[i]
    
    '''
    Animal's continent(s)
    '''
    animal_continent = [continent.getText().replace(", ", "").replace("  ", "") for continent in animal_bs("div", attrs = {"class": "col-sm-10"})[0].findAll("a")]
    animals_dictionary["Continents"] = animal_continent
    
    '''
    Animal's climate
    '''
    animal_climate = [climate.getText() for climate in animal_bs.select(".s-distr-climate a span")]
    animals_dictionary["Climate"] = animal_climate
    
    '''
    Animal's lifestyle
    '''
    animal_lifestyle = [lifestyle.getText().replace(", ", "").replace("  ", "") 
                        for lifestyle in animal_bs("div", attrs = {"class": "col-sm-9"})[0].findAll("a")]
    
    if len(animal_bs("div", attrs = {"class": "col-sm-9"})) >= 2:
        animal_lifestyle = [lifestyle.getText().replace(", ", "").replace("  ", "") 
                            for lifestyle in animal_bs("div", attrs = {"class": "col-sm-9"})[1].findAll("a")]
        
    animals_dictionary["Lifestyle"] = animal_lifestyle
    
    '''
    Animal's diet
    '''
    animal_diet = [diet.getText().replace("\n", "").replace(", ", "").replace("  ", "") for diet in animal_bs.select(".s-diet-item__link")]
    animals_dictionary["Diet"] = animal_diet
    
    '''
    Animal's population
    '''
    try:
        animal_trend = animal_bs.select(".s-population__link")[0].getText()
        animal_status = animal_bs.select(".s-population__link")[1].getText()
    
    except:
        animal_trend = None; animal_status = animal_bs.select(".s-population__link")[0].getText()
        
    finally:
        animals_dictionary["Trend"] = animal_trend
        animals_dictionary["Status"] = animal_status
        
    '''
    Animal's mating behavior
    '''
    animal_mating = None
    
    if animal_bs.select(".s-mating-slug__text")[0].getText() == "MATING BEHAVIOR":
        animal_mating = animal_bs.select(".s-mating-char")[0].getText().replace("\n", "").replace("  ", "")
  
    animals_dictionary["Mating Behavior"] = animal_mating
    
    return animals_dictionary

In [7]:
animals_list = []

for animal in animals_urls:
    try:
        animal_bs = request_soup(animal)
        animals_list.append(crawling(animal_bs))

    except:
        pass

In [9]:
'''
The f.write() method accepts only string as parameter, so we have to convert our list into json.
'''
animals_json = json.dumps(animals_list)

with open("animals'_raw.txt", mode = "w") as f:
    f.write(animals_json)